In [ ]:
from lib.models import ResNet
from lib.datasets import Dataset2p0
from torch.utils.data import DataLoader
import torch
from torch import nn
from tqdm import tqdm
from lib.utils import cms,test_evaluation
import matplotlib.pyplot as plt

In [ ]:
device = 'cuda'
model = ResNet().to(device)

params = sum([p.flatten().size()[0] for p in list(model.parameters())])
print("Params: ",params)

In [ ]:
data_dir = 'w1'
trainloader = DataLoader(Dataset2p0(dir=f'{data_dir}/train/',labels=f'{data_dir}/y_train.pt'),batch_size=64,shuffle=True)
devloader = DataLoader(Dataset2p0(dir=f'{data_dir}/dev/',labels=f'{data_dir}/y_dev.pt'),batch_size=64,shuffle=True)
testloader = DataLoader(Dataset2p0(dir=f'{data_dir}/test/',labels=f'{data_dir}/y_test.pt'),batch_size=64,shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
lossi = []

In [ ]:
model.train()
for i in range(50):
    for (X_tr,y_tr) in tqdm(trainloader):
        X_tr,y_tr = X_tr.to(device),y_tr.to(device)
        logits = model(X_tr)
        loss = criterion(logits,y_tr)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lossi.append(loss.item())

In [ ]:
plt.plot(lossi)
print(torch.tensor(lossi).mean())

In [ ]:
plt.plot(torch.tensor(lossi[100:len(lossi) - len(lossi)%20]).view(-1,20).mean(axis=1))
print(torch.tensor(lossi[100:len(lossi) - len(lossi)%20]).view(-1,20).mean(axis=1)[-1])

In [ ]:
test_evaluation(trainloader,model,criterion)
# .22

In [ ]:
test_evaluation(devloader,model,criterion)
#.22

In [ ]:
X,y = torch.load(f'{data_dir}/holdout_1.pt')

In [ ]:
from torch.utils.data import TensorDataset,DataLoader

In [ ]:
holdoutloader = DataLoader(TensorDataset(X,y),batch_size=64)

In [ ]:
import plotly.express as px
import pandas as pd
from lib.utils import get_bout_statistics_for_predictions
import numpy as np
loss,y_true,y_pred = test_evaluation(holdoutloader,model,criterion)
pred_expert = np.array(y_pred.detach().cpu()).copy()
print("Experting")
for j in range(len(pred_expert)-2):
    if((pred_expert[j:j+2]==np.array([2,0])).all()):
        pred_expert[j+1] = 2
for j in range(len(pred_expert)-2):
    if(pred_expert[j+1] != pred_expert[j] and pred_expert[j+1] != pred_expert[j+2]):
        pred_expert[j+1] = pred_expert[j]
df = pd.DataFrame([y_true.numpy(),y_pred.detach().cpu().numpy(),pred_expert],index=['true','pred','expert']).T

df.loc[df['true'] == 2,'true'] = 'W'
df.loc[df['true'] == 1,'true'] = 'S'
df.loc[df['true'] == 0,'true'] = 'P'
df.loc[df['pred'] == 2,'pred'] = 'W'
df.loc[df['pred'] == 1,'pred'] = 'S'
df.loc[df['pred'] == 0,'pred'] = 'P'
df.loc[df['expert'] == 2,'expert'] = 'W'
df.loc[df['expert'] == 1,'expert'] = 'S'
df.loc[df['expert'] == 0,'expert'] = 'P'

In [ ]:
get_bout_statistics_for_predictions(df['true'])

In [ ]:
get_bout_statistics_for_predictions(df['pred'])

In [ ]:
fig = px.line(data_frame=df)
fig.show(renderer='browser')

In [ ]:
cms(y_true=y_true,y_pred=pred_expert)